In [ ]:
#pip install plotly

import re
import datetime
import random
import numpy as np
import pandas as pd
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.preprocessing as prep
from datetime import date, timedelta , datetime
import math
import os 
from sklearn.preprocessing import OneHotEncoder 
import pickle
from datetime import datetime
import plotly.graph_objects as go


In [ ]:
data_original  = pd.read_csv('data/donors.csv' , low_memory=False)
states  = pd.read_csv('data/usa_states.csv' , sep=';')

data = data_original.copy()

data = data.rename(columns={'Unnamed: 0': 'ID'})
data.set_index('ID', inplace=True)

In [ ]:
#GENERATE RANDOM CLUSTERS FOR TESTING PURPOSE
data['CLUSTER'] = np.random.randint(0,5, size=len(data))


In [ ]:
geo = pd.DataFrame(data.groupby(['STATE','CLUSTER']).size().to_frame('SIZE') )
geo = geo.reset_index()

In [ ]:
geographic_info = pd.merge(geo, states, left_on='STATE', right_on='state')
geographic_info = geographic_info.drop(columns=['state'])
geographic_info

In [ ]:
# JUST TO SEE A DIFFERENCE BETWEEN CLUSTERS
geographic_info.loc[ (geographic_info.STATE == 'CA') & (geographic_info.CLUSTER == 1), 'SIZE'] = 60000

In [ ]:

limits = geographic_info.CLUSTER.unique().tolist()
# 5 COLORS FOR THIS SITUATION WHEN WE ONLY HAVE 5 CLUSTERS
colors = ["royalblue","crimson","lightseagreen","orange","lightgrey"]
cities = []
scale = 4

fig = go.Figure()
increaser = 0

for i in limits:    
    lim = limits[i]
    df_sub = geographic_info.loc[(geographic_info.CLUSTER == lim)]
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = df_sub['longitude'] + increaser,
        lat = df_sub['latitude'] ,
        text = df_sub[[ "name", 'SIZE']],
        marker = dict(
            size = df_sub['SIZE']/scale,
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = 'Cluster - {0} '.format(lim)))
    increaser = increaser + 0.4
    
fig.update_layout(
        title_text = 'Cluster Distribution per State',
        showlegend = True,
        geo = dict(
            scope = 'usa',
            landcolor = 'rgb(217, 217, 217)',
        )
    )

fig.show()